##Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
import os
BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/Deep-Learning-LW3'
DATA_DIR = os.path.join(BASE_DIR, "scraped_wiki_data")
DATA_FILE_DIR = os.path.join(DATA_DIR, "wiki-data-v2.csv")

In [4]:
import cv2
import albumentations as A
import glob
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import matplotlib.patches as patches
import copy
import time
import random
import csv
from skimage import io, transform
import numpy as np
import pickle
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

!pip install transformers
from transformers import ElectraModel, ElectraTokenizer, BertTokenizer, BertModel

!pip install nlu pyspark==3.0.2
import nlu
!pip install gensim
!pip install nltk

     |████████████████████████████████| 4.0 MB 8.6 MB/s 
     |████████████████████████████████| 596 kB 37.7 MB/s 
     |████████████████████████████████| 880 kB 36.3 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 41.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2c1544ecf892f410609267b3577068edc263f8983745fb64d358aab7b4982ed5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 515 kB 7.1 MB/s 
     |████████████████████████████████| 204.8 MB 56 kB/s 
     |████████████████████████████████| 198 kB 56.7 MB/s 
     |████████████████████████████████| 145 kB 53.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=5f9a0edc1973f6ad4eb2bad4415acc35a5052531d4b482d7bc36acb6df959739
  Stored in directory: /root/.cache/pip/wheels/9a/39/f6/970565f38054a830e9a8593f388b36e14d75dba6c6fdafc1ec
Successfully built pyspark


In [5]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('Found GPU :', torch.cuda.get_device_name(0))

else:
    print('Not found, use CPU instead')
    device = torch.device("cpu")

Not found, use CPU instead


##Wikipedia Api usage

In [ ]:
!pip3 install wikipedia-api
!pip install beautifulsoup4
import wikipediaapi
import random
import time
import sys
import requests
from os.path import exists
import csv
import nltk.data
nltk.download('punkt')
import seaborn as sns
from matplotlib import pyplot as plt

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13477 sha256=bf25f7316c41deee87d96b40863a90c6a58f68db152f3c3e625baaa4e0769544
  Stored in directory: /root/.cache/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
class Wikipedia_scraper:
  scraped_lines_count = 0
  dropped_count = 0
  scraped_titles = []
  
  def __init__(self, timeout_in_secs_for_exception, pages_to_scrape_count, lines_to_scrape_per_page):
    self.wiki_api = wikipediaapi.Wikipedia('en')
    self.left_to_scrape_count = pages_to_scrape_count
    self.lines_to_scrape_per_page = lines_to_scrape_per_page
    self.timeout_in_secs_for_exception = timeout_in_secs_for_exception

  def scrape_random_pages(self):
    with open(DATA_FILE_DIR, 'a') as writefile:
      csvwriter = csv.writer(writefile)
      while(self.left_to_scrape_count > 0):
        # try:
          random_title = self.get_random_wiki_page_title()
          page = self.wiki_api.page(random_title)
          print(str(self.scraped_lines_count) + ". " + str(self.dropped_count) + ". " + page.title)
          if(not self.is_page_valid(page)):
            continue
          if page.title in self.scraped_titles:
            continue
          else:
            self.add_sentences_to_file(csvwriter, page.title, page.text)
            self.scraped_titles.append(page.title)
        # except:
        #   time.sleep(self.timeout_in_secs_for_exception)
        #   continue
      writefile.close()
        
  def is_page_valid(self, page):
    if( page == None ):
      return False
    if( not page.exists() ):
      return False
    if(page.links == None):
      return False
    return True

  def get_random_wiki_page_title(self):
    wiki_url_without_title = 'https://en.wikipedia.org/wiki/'
    random_page_url = requests.get('https://en.wikipedia.org/wiki/Special:Random').url
    return random_page_url.replace(wiki_url_without_title, "")
  
  def add_sentences_to_file(self, csvwriter, title, text):
    # sentences = text.split(".")
    sentences = nltk.tokenize.sent_tokenize(text)
    sentences_to_scrape_count = min(len(sentences), self.lines_to_scrape_per_page)
    for i in range(sentences_to_scrape_count):
      if(type(sentences[i]) != str or type(title) != str):
        self.dropped_count += 1
        return
      csvwriter.writerow([sentences[i], title])
      self.scraped_lines_count += 1
      self.left_to_scrape_count -= 1

scraper = Wikipedia_scraper(60, 10000, 5)
scraper.scrape_random_pages()

0. 0. Jaws_ratio
3. 0. Kmehlener_Berge
6. 0. Cychrus_morvani
9. 0. Cong,_County_Mayo
14. 0. Karan_Patel
19. 0. Orville_E._Babcock
24. 0. Man%27s_Best_Friend_(album)
24. 0. John_Maggio_(director)
29. 0. John_Pezzenti
34. 0. Karen_Dunbar
39. 0. Abdullah_Qureshi_(artist)
44. 0. Chiefdom_of_Mangshi
49. 0. John_A._and_Martha_Robertson_House
54. 0. Danny_Ragsdale
59. 0. Arlene_Alda
64. 0. Ernest_Ng%27eno
64. 0. Farmacias_Benavides
69. 0. RKSV_Scherpenheuvel
74. 0. Death_Chorus
78. 0. Gardinia
83. 0. Xiyunykus
88. 0. 1898_Colgate_football_team
91. 0. Thomas_Buck_Reed
96. 0. John_P._Jaeckel
101. 0. Janet_Reed
106. 0. Troilus_luridus
111. 0. Andrew_of_Cicala
116. 0. Siege_of_Fort_at_Number_4
121. 0. Vespers
126. 0. Backstreet_(album)
130. 0. Lizard_Pond
132. 0. Chad_Donella
137. 0. List_of_Bizaardvark_episodes
142. 0. Tharon_Drake
147. 0. Dan_Morgan_(footballer)
149. 0. S%C3%A3o_Jo%C3%A3o_do_Arraial
149. 0. Anni_Pede-Erdkamp
154. 0. New_towns_of_Hong_Kong
159. 0. Elizabeth_Gage
164. 0. %C7%A2rr

KeyboardInterrupt: ignored

##GMM

###Dataset

In [6]:
col_names = ["sentence", "title"]

df = pd.read_csv(DATA_FILE_DIR, 
                delimiter=',' , 
                header= None,
                names= col_names)



In [7]:
def drop_all_invalid_sentences(df):
  # done = False

  # while(not done):
  #   done = True
  #   for i, sentence in enumerate(df.sentence):
  #     if(type(sentence) != str or len(sentence.split()) > 255):
  #       df = df.drop(df.index[i])
  #       done = False
  #       break
  # return df
  
  done = False

  while(not done):
    done = True
    for i, sentence in enumerate(df.sentence):
      #  or len(sentence.split()) > 255
      if(type(sentence) != str):
        df = df.drop(df.index[i])
        done = False
        break
  return df

df = drop_all_invalid_sentences(df)

###----------

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
from transformers import ElectraModel, ElectraConfig

configuration = ElectraConfig()
model = ElectraModel(configuration)
configuration = model.config

In [13]:
from transformers import ElectraTokenizer, ElectraModel
import torch

tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
model = ElectraModel.from_pretrained("google/electra-small-discriminator")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
def encode(string):
  inputs = tokenizer(string, return_tensors="pt")
  if(len(inputs.input_ids[0]) >= 512):
    return None
  return model(**inputs)[0].detach().squeeze()

data = []
outputs_done = 0
for sentence, title in zip(df.sentence, df.title):
  encoded = encode(sentence)
  if(encoded != None):
    new_row = type('new_row', (object,), {
        'sentence': sentence,
        "title": title,
        "encoded": encoded
    })()
    data.append(new_row)
    outputs_done += 1
  if (outputs_done == 1500):
    break

len(data)

1500

In [70]:
from scipy import spatial

test_sentence = "Johny was a good boy"
encoded_test_sentence = encode(test_sentence)


min_diff = 1
for row in data:
  # print(row.encoded[0])
  # print(encoded_test_sentence[0])
  # break
  diff = spatial.distance.cosine(row.encoded[0], encoded_test_sentence[0])
  if(diff < min_diff):
    min_diff = diff
    pred_row = row
print(min_diff)
print(pred_row.sentence)

0.09078001976013184
Suddenly Misha falls into the river, but was saved by a stray dog, who after that got to the furrier.


In [57]:
data[102].sentence

'In 1951, ministries were established in Japan and the Dominican Republic.'